In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import joblib
from statsmodels.tsa.arima.model import ARIMA

# Paths
CSV_PATH = r"C:/Users/hp/Desktop/Shalini - Project/Dataset.csv"
GBM_MODEL_PATH = "gbm_timeseries_model.joblib"
SCALER_PATH = "feature_scaler.joblib"
TARGET = "energy_consumption"

# Load Data
df = pd.read_csv(CSV_PATH, parse_dates=["timestamp"])
df = df.sort_values("timestamp").reset_index(drop=True)

# Features
def create_time_features(df):
    df["hour"] = df["timestamp"].dt.hour
    df["dayofweek"] = df["timestamp"].dt.dayofweek
    df["day"] = df["timestamp"].dt.day
    df["month"] = df["timestamp"].dt.month
    return df

df = create_time_features(df)

# Lags
LAGS = list(range(1, 25))
for lag in LAGS:
    df[f"lag_{lag}"] = df[TARGET].shift(lag)

df["roll_mean_24"] = df[TARGET].rolling(24).mean().shift(1)
df["roll_mean_168"] = df[TARGET].rolling(168).mean().shift(1)

df_model = df.dropna().reset_index(drop=True)

# Train/test split
test_days = 180
test_start = df_model["timestamp"].max() - pd.Timedelta(days=test_days)
train_df = df_model[df_model["timestamp"] < test_start]
test_df = df_model[df_model["timestamp"] >= test_start]

feature_cols = [
    "temperature", "humidity", "solar_radiation", "wind_speed",
    "hour", "dayofweek", "day", "month"
] + [f"lag_{lag}" for lag in LAGS] + ["roll_mean_24", "roll_mean_168"]

X_train = train_df[feature_cols].values
y_train = train_df[TARGET].values
X_test = test_df[feature_cols].values
y_test = test_df[TARGET].values

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
joblib.dump(scaler, SCALER_PATH)

# Train GBM
gbm = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    random_state=42
)
gbm.fit(X_train_scaled, y_train)

# Save Model
joblib.dump(gbm, GBM_MODEL_PATH)

print("Model and scaler saved successfully!")
print("Ready to run SHAP now.")
